# 🧠 Урок 16: Простая нейросеть с нуля (на NumPy)
**Цель урока:** Познакомиться с устройством нейросети, реализовать простейшую модель на NumPy, понять, как обучается сеть через градиентный спуск. Подходит для новичков.

## 📌 Что такое нейросеть?
- **Нейрон** — базовый элемент сети, который:
  1. Принимает входные данные (например, `x1`, `x2`).
  2. Умножает их на веса (например, `w1`, `w2`).
  3. Суммирует результаты: `z = w1*x1 + w2*x2 + b`.
  4. Применяет функцию активации: `a = sigmoid(z)`.
- **Слои:**
  - **Входной слой:** Принимает данные.
  - **Скрытый слой:** Обрабатывает данные.
  - **Выходной слой:** Возвращает результат.
- **Веса:** Параметры, которые сеть корректирует в процессе обучения.
- **Функция активации:** Добавляет нелинейность, например, сигмоида или ReLU.

💡 **Аналогия:** Нейросеть — как мозг человека: нейроны связаны между собой и передают сигналы, чтобы принять решение .

## 🧱 Как обучается нейросеть?
### 1. Прямое распространение (Forward Pass)
- Данные проходят через сеть от входного к выходному слою.
- Каждый нейрон вычисляет взвешенную сумму входов и применяет функцию активации.

**Пример для скрытого слоя:**
```python
hidden_input = X @ weights_input_hidden  # Матричное умножение
hidden_output = sigmoid(hidden_input)  # Применение сигмоиды
```

**Пример для выходного слоя:**
```python
output_input = hidden_output @ weights_hidden_output
predicted = sigmoid(output_input)
```

### 2. Функция потерь (Loss Function)
- **MSE (среднеквадратичная ошибка):** Используется для регрессии.
  ```python
  loss = np.mean((y_true - y_pred)**2)
  ```
- **Cross-Entropy:** Для классификации.

### 3. Обратное распространение (Backpropagation)
- **Шаг 1:** Вычислите ошибку выходного слоя: `output_error = y - predicted`.
- **Шаг 2:** Вычислите градиенты через цепное правило:
  ```python
  output_delta = output_error * sigmoid_derivative(predicted)
  hidden_error = output_delta @ weights_hidden_output.T
  hidden_delta = hidden_error * sigmoid_derivative(hidden_output)
  ```
- **Шаг 3:** Обновите веса:
  ```python
  weights_hidden_output += hidden_output.T @ output_delta
  weights_input_hidden += X.T @ hidden_delta
  ```

### 4. Градиентный спуск
- **Идея:** Минимизировать ошибку, двигаясь в направлении убывания градиента.
- **Формула обновления весов:**
  ```python
  weights -= learning_rate * gradients
  ```
- **learning_rate:** Шаг обучения (слишком большой — обучение не сойдётся, слишком маленький — медленно).

💡 **Почему это работает?** Нейросеть сама определяет, какие признаки важны для задачи, итеративно корректируя веса .

## 🧪 Практика: Реализация нейросети на NumPy
### Шаг 1: Подготовка данных
Будем обучать сеть решать задачу логического AND:

In [ ]:
import numpy as np

# Входные данные (X) и целевые значения (y) для логического AND
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [0], [0], [1]])
print("Входные данные:", X)
print("Целевые значения:", y)

### Шаг 2: Инициализация весов и функций активации

In [ ]:
# Сигмоида — функция активации
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Производная сигмоиды для backpropagation
def sigmoid_derivative(x):
    return x * (1 - x)

# Инициализация весов случайными числами
np.random.seed(42)
weights_input_hidden = np.random.rand(2, 4)  # Входной -> скрытый слой
weights_hidden_output = np.random.rand(4, 1)  # Скрытый -> выходной слой
print("Веса вход-скрытый:", weights_input_hidden)
print("Веса скрытый-выход:", weights_hidden_output)

### Шаг 3: Прямое распространение (Forward Pass)

In [ ]:
def forward_pass(X):
    # Скрытый слой
    hidden_layer_input = np.dot(X, weights_input_hidden)
    hidden_layer_output = sigmoid(hidden_layer_input)
    
    # Выходной слой
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
    predicted = sigmoid(output_layer_input)
    
    return hidden_layer_output, predicted

# Пример прямого прохода
hidden, output = forward_pass(X)
print("Предсказания до обучения:", output)

### Шаг 4: Обратное распространение (Backpropagation)

In [ ]:
def backward_pass(X, y, hidden, predicted):
    # Ошибка выходного слоя
    output_error = y - predicted
    output_delta = output_error * sigmoid_derivative(predicted)
    
    # Ошибка скрытого слоя
    hidden_error = output_delta @ weights_hidden_output.T
    hidden_delta = hidden_error * sigmoid_derivative(hidden)
    
    # Обновление весов
    global weights_input_hidden, weights_hidden_output
    weights_hidden_output += hidden.T @ output_delta
    weights_input_hidden += X.T @ hidden_delta

# Обучение
losses = []
learning_rate = 0.1
for epoch in range(10000):
    hidden, predicted = forward_pass(X)
    loss = np.mean((y - predicted)**2)
    losses.append(loss)
    backward_pass(X, y, hidden, predicted)

print("Предсказания после обучения:", predicted)

### Шаг 5: Визуализация ошибки по эпохам

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.title('Ошибка по эпохам')
plt.xlabel('Эпохи')
plt.ylabel('MSE')
plt.grid(True)
plt.show()

## 📝 Домашнее задание
**Задача 1:** Измените архитектуру сети — увеличьте число нейронов во скрытом слое до 5. Как это повлияет на обучение?
**Задача 2:** Замените сигмоиду на ReLU: `f(x) = max(0, x)` и `f'(x) = 1 if x > 0 else 0`. Сравните результаты.

## ✅ Рекомендации по выполнению
- **Задача 1:** Увеличьте размер скрытого слоя, измените `weights_input_hidden` на `(2, 5)` и `weights_hidden_output` на `(5, 1)`.
- **Задача 2:** Реализуйте ReLU и запустите обучение. Проверьте, как быстро сеть сходится и какова ошибка.